<a href="https://colab.research.google.com/github/alisdghnia/rnn-lstm-interest-rate-prediction/blob/main/RNN_LSTM_InterestRate_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
uploaded = files.upload()

In [39]:
ir = pd.read_excel('IR-data.xlsx')
ir = ir.rename(columns={'Interest rate': 'interest_rate'})
ir = ir.set_index('Date')
ir

,interest_rate
Date,
2010-12-30,4.20
2011-01-06,4.13
2011-01-13,4.08
2011-01-20,4.05
2011-01-27,4.09
...,...
2022-02-24,3.14
2022-03-03,3.01
2022-03-10,3.09


In [48]:
def create_sequences(data, window_size):
    train_window = []
    target_window = []
    for i in range(len(data) - window_size):
        train_window.append(data.iloc[i:i+window_size-1, :])
        target_window.append(data.iloc[i+window_size, :])

    return train_window, target_window

In [82]:
train = ir[ir.index < '2021-01-01']
# create a test data with 50 more data points for predictions
# might want to think about just leaving out 50 data points between train and test
# this will prevent the possible data leakage that is happening here
# otherwise only 14 rows of data will be predicted with 50 window size
start_date = pd.to_datetime('2021-01-01') - pd.DateOffset(days=350)
test = ir[ir.index >= start_date]
# test = ir[ir.index >= '2021-01-01']

x_train, y_train = create_sequences(train, window_size= 50)
x_test, y_test = create_sequences(test, window_size= 50)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((473, 49, 1), (473, 1), (64, 49, 1), (64, 1))

# RNN Application

In [83]:
window_size = 50
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(x_train, y_train, epochs=20)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 6.8718
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4244
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0722
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0310
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0163
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0130
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0150
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0128
Epoch 9/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0174
Epoch 10/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0129
Epoch 11/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0126
Epoch 12/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0142
Epoch 13/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0136
Epoch 14/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0130
Epoch 15/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0140
Epoch 16/20
15/

In [86]:
# start with the first sequence from the test data
last_sequence = np.array(test['interest_rate'])[:window_size].reshape(-1, 1)  # reshape to make sure it's 2D

predictions = []

# use the 50 + len(ir[ir.index >= '2021-01-01']) data that we assigned earlier
# to predict for the entire length of the ir[ir.index >= '2021-01-01'] data
for i in range(len(test) - window_size):
    prediction = model.predict(last_sequence.reshape(1, window_size, 1))
    predictions.append(prediction[0, 0])

    # reshape the next value to be appended
    next_value = np.array(test['interest_rate'])[i + window_size].reshape(-1, 1)

    last_sequence = np.append(last_sequence[1:], next_value, axis=0)

print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━

In [91]:
final_df = ir[ir.index >= '2021-01-01']
final_df.loc[:, 'interest_rate_rnn'] = predictions
final_df = final_df.rename(columns={'interest_rate': 'interest_rate_original'})
final_df

<ipython-input-91-4023a06a1266>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.loc[:, 'interest_rate_rnn'] = predictions


,interest_rate_original,interest_rate_rnn
Date,,
2021-01-07,2.16,2.257776
2021-01-14,2.23,2.236353
2021-01-21,2.21,2.232058
2021-01-28,2.20,2.244044
2021-02-04,2.21,2.243789
...,...,...
2022-02-24,3.14,2.921634
2022-03-03,3.01,3.047959
2022-03-10,3.09,3.098802


In [92]:
rnn_mse = np.mean((final_df['interest_rate_rnn'] - final_df['interest_rate_original'])**2)
rnn_mse

0.01637066414801336

# LSTM Application

In [ ]:
window_size = 50
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(x_train, y_train, epochs=20)

In [93]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=20)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 9.5750
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.5514
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0628
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0376
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0315
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0295
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0257
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0286
Epoch 9/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0266
Epoch 10/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0250
Epoch 11/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0237
Epoch 12/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0240
Epoch 13/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0237
Epoch 14/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0219
Epoch 15/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0184
Epoch 16/20
15/

In [94]:
# start with the first sequence from the test data
last_sequence = np.array(test['interest_rate'])[:window_size].reshape(-1, 1)  # reshape to make sure it's 2D

lstm_predictions = []

# use the 50 + len(ir[ir.index >= '2021-01-01']) data that we assigned earlier
# to predict for the entire length of the ir[ir.index >= '2021-01-01'] data
for i in range(len(test) - window_size):
    prediction = model.predict(last_sequence.reshape(1, window_size, 1))
    lstm_predictions.append(prediction[0, 0])

    # reshape the next value to be appended
    next_value = np.array(test['interest_rate'])[i + window_size].reshape(-1, 1)

    last_sequence = np.append(last_sequence[1:], next_value, axis=0)

print(lstm_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━

In [95]:
final_df.loc[:, 'interest_rate_lstm'] = lstm_predictions
final_df

,interest_rate_original,interest_rate_rnn,interest_rate_lstm
Date,,,
2021-01-07,2.16,2.257776,2.330104
2021-01-14,2.23,2.236353,2.310937
2021-01-21,2.21,2.232058,2.302372
2021-01-28,2.20,2.244044,2.296935
2021-02-04,2.21,2.243789,2.290770
...,...,...,...
2022-02-24,3.14,2.921634,2.630768
2022-03-03,3.01,3.047959,2.712975
2022-03-10,3.09,3.098802,2.782902


In [96]:
lstm_mse = np.mean((final_df['interest_rate_lstm'] - final_df['interest_rate_original'])**2)
rnn_mse = np.mean((final_df['interest_rate_rnn'] - final_df['interest_rate_original'])**2)
lstm_rnn_mse = np.mean((final_df['interest_rate_lstm'] - final_df['interest_rate_rnn'])**2)
print(f'lstm_mse: {lstm_mse}')
print(f'rnn_mse: {rnn_mse}')
print(f'lstm_rnn_mse: {lstm_rnn_mse}')

lstm_mse: 0.04351921649338991
rnn_mse: 0.01637066414801336
lstm_rnn_mse: 0.01449275016784668
